# Assignment: Text Classification


## Kaggle Competition: https://www.kaggle.com/c/nlp-getting-started/overview

Twitter has become an important communication channel in times of emergency.
The ubiquitousness of smartphones enables people to announce an emergency they’re observing in real-time. Because of this, more agencies are interested in programatically monitoring Twitter (i.e. disaster relief organizations and news agencies).

In this competition, you’re challenged to build a machine learning model that predicts which Tweets are about real disasters and which one’s aren’t.

### Dataset
Each sample in the train and test set has the following information:

* The text of a tweet
* A keyword from that tweet (although this may be blank!)
* The location the tweet was sent from (may also be blank)



## Task 1: Problem Statement
Discuss the problem setting and the first implcations of the given data set... 
* What assumptions can we make about the data?
* What problems are we expecting?

## Task 2: Feature Extraction
This week we focus on the extraction of text features. you are free to choose a ***word2vec*** or ***BERT*** embedding (bert is harder to setup but gives better results).

### word2vec / doc2vec
* use the implementation of the  **gensim** library

#### API
* https://radimrehurek.com/gensim/models/word2vec.html
* https://radimrehurek.com/gensim/models/doc2vec.html


### Bert
* kaggle tutorial on pre-trained Bert usage: https://keras.io/examples/nlp/text_extraction_with_bert/
* Note: the fine tuning step can be compute intensive, for this assignment it is ok to just use the pre-trained model


In [1]:
import os
import re
import json
import string
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tokenizers import BertWordPieceTokenizer
from transformers import BertTokenizer, TFBertModel, BertConfig

max_len = 384
configuration = BertConfig()  # default parameters and configuration for BERT

In [2]:
data = pd.read_csv('train.csv' , encoding = "ISO-8859-1" )

In [3]:
data.head(20)

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
5,8,NaN,NaN,#RockyFire Update => California Hwy. 20 closed...,1
6,10,NaN,NaN,#flood #disaster Heavy rain causes flash flood...,1
7,13,NaN,NaN,I'm on top of the hill and I can see a fire in...,1
8,14,NaN,NaN,There's an emergency evacuation happening now ...,1
9,15,NaN,NaN,I'm afraid that the tornado is coming to our a...,1


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        7613 non-null   int64 
 1   keyword   7552 non-null   object
 2   location  5080 non-null   object
 3   text      7613 non-null   object
 4   target    7613 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 297.5+ KB


In [5]:
data["target"].unique()

array([1, 0], dtype=int64)

In [6]:
# Save the slow pretrained tokenizer
#slow_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
#save_path = "bert_base_uncased/"
#if not os.path.exists(save_path):
#    os.makedirs(save_path)
#slow_tokenizer.save_pretrained(save_path)

# Load the fast tokenizer from saved file
#tokenizer = BertWordPieceTokenizer("bert_base_uncased/vocab.txt", lowercase=True)

In [7]:
from sklearn.model_selection import train_test_split

X = data.text.values
y = data.target.values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=42)

In [8]:
import nltk
# Uncomment to download "stopwords"
nltk.download("stopwords")
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Felix\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [9]:
def text_preprocessing(s):
    """
    - Lowercase the sentence
    - Change "'t" to "not"
    - Remove "@name"
    - Isolate and remove punctuations except "?"
    - Remove other special characters
    - Remove stop words except "not" and "can"
    - Remove trailing whitespace
    """
    s = s.lower()
    # Change 't to 'not'
    s = re.sub(r"\'t", " not", s)
    # Remove @name
    s = re.sub(r'(@.*?)[\s]', ' ', s)
    # Isolate and remove punctuations except '?'
    s = re.sub(r'([\'\"\.\(\)\!\?\\\/\,])', r' \1 ', s)
    s = re.sub(r'[^\w\s\?]', ' ', s)
    # Remove some special characters
    s = re.sub(r'([\;\:\|•«\n])', ' ', s)
    # Remove stopwords except 'not' and 'can'
    s = " ".join([word for word in s.split()
                  if word not in stopwords.words('english')
                  or word in ['not', 'can']])
    # Remove trailing whitespace
    s = re.sub(r'\s+', ' ', s).strip()
    
    return s

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
# Preprocess text
X_train_preprocessed = np.array([text_preprocessing(text) for text in X_train])
X_test_preprocessed = np.array([text_preprocessing(text) for text in X_test])

# Calculate TF-IDF
tf_idf = TfidfVectorizer(ngram_range=(1, 3), binary=True, smooth_idf=False)
X_train_tfidf = tf_idf.fit_transform(X_train_preprocessed)
X_test_tfidf = tf_idf.transform(X_test_preprocessed)

In [11]:
from sklearn.model_selection import StratifiedKFold, cross_val_score

def get_auc_CV(model):
    """
    Return the average AUC score from cross-validation.
    """
    # Set KFold to shuffle data before the split
    kf = StratifiedKFold(5, shuffle=True, random_state=1)

    # Get AUC scores
    auc = cross_val_score(
        model, X_train_tfidf, y_train, scoring="roc_auc", cv=kf)

    return auc.mean()

## Task 3: Train a  Model
* Which ML model would you choose and why?
* Train and evaluate the model using the train data
* Is the data blanced? What are the implications, how can you deal with this?
* Discuss the results -> possible improvements?


In [15]:
from sklearn.naive_bayes import MultinomialNB

res = pd.Series([get_auc_CV(MultinomialNB(i))
                 for i in np.arange(1, 10, 0.1)],
                index=np.arange(1, 10, 0.1))

best_alpha = np.round(res.idxmax(), 2)
print('Best alpha: ', best_alpha)


C:\Users\Felix\anaconda3\lib\site-packages\sklearn\utils\validation.py:68: FutureWarning: Pass alpha=1.0 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "
C:\Users\Felix\anaconda3\lib\site-packages\sklearn\utils\validation.py:68: FutureWarning: Pass alpha=1.1 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "
C:\Users\Felix\anaconda3\lib\site-packages\sklearn\utils\validation.py:68: FutureWarning: Pass alpha=1.2000000000000002 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "
C:\Users\Felix\anaconda3\lib\site-packages\sklearn\utils\validation.py:68: FutureWarning: Pass alpha=1.3000000000000003 as keyword args. From version 0.25 passing these as positional arguments 

Best alpha:  1.7


C:\Users\Felix\anaconda3\lib\site-packages\sklearn\utils\validation.py:68: FutureWarning: Pass alpha=9.800000000000008 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "
C:\Users\Felix\anaconda3\lib\site-packages\sklearn\utils\validation.py:68: FutureWarning: Pass alpha=9.900000000000007 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "


In [16]:
# Compute predicted probabilities
nb_model = MultinomialNB(alpha=1.8)
nb_model.fit(X_train_tfidf, y_train)
probs = nb_model.predict_proba(X_test_tfidf)
nb_predict = nb_model.predict(X_test_tfidf)

## Task 4: Evaluate 
* the test data does not contain labels! -> You can upload your solution to kaggle to get a score or build your own test set by splitting the training data

In [17]:
from sklearn.metrics import classification_report

print(classification_report(y_test, nb_predict))

              precision    recall  f1-score   support

           0       0.75      0.94      0.84      1318
           1       0.88      0.58      0.70       966

    accuracy                           0.79      2284
   macro avg       0.82      0.76      0.77      2284
weighted avg       0.81      0.79      0.78      2284

